In [6]:
import cv2
import csv
import numpy as np
from os import listdir, makedirs, path, link
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib import pylab
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
%matplotlib inline
pylab.rcParams['figure.figsize'] = (150, 50)



In [2]:

trf = 'Scan'  # subfolder scans
root_folder = Path('/home/simone/development/python_università/Mario/RAW DATA')
ext = '.tif'  # source file extention
ext_save = '.tif'  # save file extention
root_folder_save = Path('/home/simone/development/python_università/fruit_detection/dataset_classification')
subdirs = [x.name for x in root_folder.iterdir() if x.is_dir()]

In [3]:
method = widgets.Dropdown(
    options=['BINARY', 'BINARY+OTSU', 'ADAPTIVE_MEAN', 'ADAPTIVE_GAUSSIAN'],
    value='BINARY+OTSU',
    description='Method',
    disabled=False)

dir_selection = widgets.Dropdown(
    options=subdirs,
    value=subdirs[0],
    description='Folder',
    disabled=False
    )
file_selection = widgets.Dropdown(
    options=[each for each in listdir(root_folder / dir_selection.value) if each.endswith(ext)],
    description='Scan',
    disabled=False)
def update_file_list(folder):
    new_folder = root_folder / str(folder['new'])
    file_selection.options = [each for each in listdir(new_folder) if each.endswith(ext)]
dir_selection.observe(update_file_list, names='value')  #update file list on folder change giving the new value to update_file_list
cut_left = widgets.IntSlider(value = 5,
                             min = 0,
                             max = 50,
                             step = 1,
                             description = 'Cut_left',
                             continuous_update = False)
cut_right = widgets.IntSlider(value = 95,
                              min = 50,
                              max = 100,
                              step = 1,
                              description = 'Cut_right',
                              continuous_update = False)
cut_top = widgets.IntSlider(value = 5,
                            min = 0,
                            max = 50,
                            step = 1,
                            description = 'Cut_top',
                            continuous_update = False)
cut_bottom = widgets.IntSlider(value = 95,
                               min = 50,
                               max = 100,
                               step = 1,
                               description = 'Cut_bottom',
                               continuous_update = False)
threshold = widgets.IntSlider(value = 100,
                              min = 0,
                              max = 255,
                              step = 1,
                              description = 'Global thr',
                              continuous_update = False)
denoise = widgets.IntSlider(value = 3,
                            min = 0,
                            max = 100,
                            step = 1,
                            description = 'Denoise',
                            continuous_update = False)
min_area = widgets.IntSlider(value = 3,
                            min = 0,
                            max = 100,
                            step = 1,
                            description = 'Min Area (in %)',
                            continuous_update = False)
max_area = widgets.IntSlider(value = 50,
                            min = 0,
                            max = 100,
                            step = 1,
                            description = 'Max Area (in %)',
                            continuous_update = False)
blocksize = widgets.IntText(value = 101,
                            min = 1,
                            max = 201,
                            step = 2,
                            description = 'Blocksize',
                            continuous_update = False)
#progess_bar = widgets.IntProgress(
#    value=0,
#    min=0,
#    max=10,
#    step=1,
#    description='Loading:',
#    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
#    orientation='horizontal'
#)

save_files = widgets.ToggleButton(
    value=False,
    description='Save images',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

def show_widget_on_method(change):
    methos = change['new']
    print(f"somethig changed to {method}")
    if 'BINARY' in method :
        threshold.layout.visibility = 'visible'
        blocksize.layout.visibility = 'hidden'
    elif 'ADAPTIVE' in method:
        threshold.layout.visibility = 'hidden'
        
        blocksize.layout.visibility = 'visible'
        
method.observe(show_widget_on_method, names='value') 

def update_file_list(folder):
    new_folder = root_folder / str(folder['new'])
    file_selection.options = [each for each in listdir(new_folder) if each.endswith(ext)]
    
dir_selection.observe(update_file_list, names='value')  #update file list on folder change giving the new value to update_file_list
widget_conf = {
     'dir_selection': dir_selection,
     'file_selection': file_selection,
     'method': method,
     'min_area': min_area,
     'max_area': max_area,
     'cut_left': cut_left,
     'cut_right': cut_right,
     'cut_top': cut_top,
     'cut_bottom': cut_bottom,
     'threshold': threshold,
     'denoise': denoise,
     'blocksize': blocksize,
     'plot': False,
     'plot_extracted': False,
     'save_files': save_files 
}

In [4]:
def segmentation_func(dir_selection, file_selection, cut_left, cut_right, cut_top, cut_bottom, method, min_area, max_area, threshold, denoise, blocksize, plot, plot_extracted, save_files):
    print(f"processing: {locals()}")
    # must split this function in sub function
    fullname_fg = root_folder / dir_selection / file_selection #join paths using pathlib to abstract OS differences
    img = cv2.imread(str(fullname_fg))
    img_cut = img.copy()
    h, w = img_cut.shape[:2]
    max_area = int(w * max_area / 100) ** 2
    min_area = int(w * min_area / 100) ** 2
    
    w_left = int(w*cut_left/100)
    cv2.line(img_cut,(w_left, 0),(w_left, h),(0,255,0), int(w*0.01))
    w_right = int(w*cut_right/100)
    cv2.line(img_cut,(w_right, 0),(w_right, h),(255,255,0), int(w*0.01))
    h_top = int(h*cut_top/100)
    cv2.line(img_cut,(0, h_top),(w, h_top),(0,0,255), int(w*0.01))
    h_bottom = int(h*cut_bottom/100)
    cv2.line(img_cut,(0, h_bottom),(w, h_bottom),(255,0,255), int(w*0.01))
    img = img[h_top:h_bottom, w_left:w_right,:]
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    if method == 'BINARY':
        ret, th1 = cv2.threshold(gray,threshold,255,cv2.THRESH_BINARY)
    if method == 'BINARY+OTSU':
        ret, th1 = cv2.threshold(gray,threshold,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    if method == 'ADAPTIVE_MEAN':
        th1 = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blocksize, 0)
    if method == 'ADAPTIVE_GAUSSIAN':
        gray = cv2.GaussianBlur(gray,(5,5),0)
        th1 = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, blocksize, 0)

    #floodfill
    th1_floodfill = th1.copy()
    hf, wf = th1.shape[:2]
    mask = np.zeros((hf + 2, wf + 2), np.uint8)
    cv2.floodFill(th1_floodfill, mask, (0, 0), 255)
    th1_floodfill_inv = cv2.bitwise_not(th1_floodfill)
    th1 = th1 | th1_floodfill_inv


    #denoise
    kernel = np.ones((denoise,denoise),np.uint8)
    th1 = cv2.morphologyEx(th1,cv2.MORPH_OPEN,kernel, iterations = 2)



    _, contours, hi = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #get countours
    #writer countors to image
    img_contours = img.copy()

    cropped_images = []
    n_samples = 0
    for i, cnt in enumerate(contours):
        X, Y, w, h = cv2.boundingRect(cnt)
        area = cv2.contourArea(cnt)

        if (area < min_area or area > max_area):
            continue

        section = img[Y:Y + h, X:X + w]
        mask = th1[Y:Y + h, X:X + w]
        
        cv2.drawContours(img_contours, contours, i, (0, 255, 0), 3)
        cropped_images.append((section, mask))
        n_samples += 1


    if plot == True:
        images = [img_cut[:,:,::-1], th1, img_contours[:,:,::-1]]
        colors = [None, 'gray', None]
        if plot_extracted:
            n_rows = (len(cropped_images) // 3) + 3 # to ensure enogh space for cropped images plus orginal images
            for idx in range(len(cropped_images)):
                plt.subplot(n_rows ,3,3 + idx + 1),plt.imshow(cropped_images[idx][0][:,:,::-1])
        else:
            for i in range(3):
                plt.subplot(1,3,i+1),plt.imshow(images[i], colors[i])
        plt.show()

    print("Detected samples:", str(n_samples))
    if save_files:
        folder_save = root_folder_save / dir_selection
        print(f"saving to files to {folder_save}")
        try:
            makedirs(folder_save)
        except:
            pass
        for i in range(len(cropped_images)):
            image, mask = cropped_images[i]
            #dirty image
            #filename_image_dirty = folder_save / f"{file_selection[:-len(ext)]}_{i}_dirty{ext_save}" #generate file name
            #cv2.imwrite(str(filename_image_dirty), image) #images
            # mask
            #filename_mask = folder_save / f"{file_selection[:-len(ext)]}_{i}_mask_{ext_save}"
            #cv2.imwrite(str(filename_mask), mask)
            # inverted mask
            #filename_mask = folder_save / f"{file_selection[:-len(ext)]}_{i}_mask_inv{ext_save}"
            #mask_inv = cv2.bitwise_not(mask)
            #cv2.imwrite(str(filename_mask), mask_inv)
            # clean image
            filename_image_clean = folder_save / f"{file_selection[:-len(ext)]}_{i}_clean{ext_save}"
            image_clean = image & \
                np.stack([mask, mask, mask], axis=2) #to make background perfectly black
            cv2.imwrite(str(filename_image_clean), image_clean)
            #background only
            #filename_background = folder_save / f"{file_selection[:-len(ext)]}_{i}_background{ext_save}"
            #image_background = image | \
                #np.stack([mask, mask, mask], axis=2) #to extract background
            #cv2.imwrite(str(filename_background), image_background)
            print(f"saved file: {filename_image_clean}")
            

In [8]:

interact(segmentation_func, **widget_conf) # using widget_conf paramenter unpacking to simply this cell

interactive(children=(Dropdown(description='Folder', options=('Cetriolo', 'Peperoncino', 'Patata', 'Pomodoro -…

<function __main__.segmentation_func(dir_selection, file_selection, cut_left, cut_right, cut_top, cut_bottom, method, min_area, max_area, threshold, denoise, blocksize, plot, plot_extracted, save_files)>

In [7]:
folder = 'Pomodoro - Ciliegino'
def save_folder(folder):
    
    option = {'dir_selection': folder,
              'file_selection': 'cetrioli2.tif', 'cut_left': 5,
              'cut_right': 95, 'cut_top': 5, 'cut_bottom': 95, 
              'method': 'BINARY+OTSU', 'min_area': 3, 'max_area': 50,
              'threshold': 100, 'denoise': 3, 'blocksize': 101,
              'plot': False, 'plot_extracted': False, 'save_files': True}
    for i in [each for each in listdir(root_folder / folder) if each.endswith(ext)]:
        option['file_selection'] = i
        segmentation_func(**option)
save_folder(folder)

processing: {'dir_selection': 'Pomodoro - Ciliegino', 'file_selection': 'pomodoro2.tif', 'cut_left': 5, 'cut_right': 95, 'cut_top': 5, 'cut_bottom': 95, 'method': 'BINARY+OTSU', 'min_area': 3, 'max_area': 50, 'threshold': 100, 'denoise': 3, 'blocksize': 101, 'plot': False, 'plot_extracted': False, 'save_files': True}
Detected samples: 24
saving to files to /home/simone/development/python_università/fruit_detection/dataset_classification/Pomodoro - Ciliegino
saved file: /home/simone/development/python_università/fruit_detection/dataset_classification/Pomodoro - Ciliegino/pomodoro2_0_clean.tif
saved file: /home/simone/development/python_università/fruit_detection/dataset_classification/Pomodoro - Ciliegino/pomodoro2_1_clean.tif
saved file: /home/simone/development/python_università/fruit_detection/dataset_classification/Pomodoro - Ciliegino/pomodoro2_2_clean.tif
saved file: /home/simone/development/python_università/fruit_detection/dataset_classification/Pomodoro - Ciliegino/pomodoro2_3_

!! questa cella è segnata come markdown e non codice!
for fname in flist:
    th1, img = segmentation_func(fname,
                            cut_left.value,
                            cut_right.value,
                            cut_top.value,
                            cut_bottom.value,
                            method.value,
                            threshold.value,
                            denoise.value,
                            blocksize.value,
                            plot = False)
    _, contours, hi = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img_contours = img.copy()
    for i, cnt in enumerate(contours):
        X, Y, w, h = cv2.boundingRect(cnt)
        cv2.drawContours(img_contours, contours, i, (0, 255, 0), 3)
        section = img[Y:Y + h, X:X + w]
        path_name = "./Results/Sample_" + str(fname[:-len(ext)])
        if not path.exists(path_name):
            makedirs(path_name)
        cv2.imwrite(path_name + "/" + str(fname[:-len(ext)]) + "_crop_" + str(i) + ext_save, section)
    cv2.imwrite("./Results/" + str(fname[:-len(ext)]) + '_contours' + ext_save, img_contours)
    cv2.imwrite("./Results/" + str(fname[:-len(ext)]) + '_mask' + ext_save, th1)

In [8]:
all_folders = ['Cetriolo', 'Peperoncino', 'Patata', 'Pomodoro - San Marzano', 'Banana', 'Carota', 'Albicocca',
              'Kiwi', 'Zucchino', 'Pera', 'Mela', 'Lime', 'Champignon',
              'Prugna rossa', 'Pesca', 'Cipolla', 'Pomodoro - Ciliegino']
for folder in all_folders:
    save_folder(folder)

processing: {'dir_selection': 'Cetriolo', 'file_selection': 'cetrioli4.tif', 'cut_left': 5, 'cut_right': 95, 'cut_top': 5, 'cut_bottom': 95, 'method': 'BINARY+OTSU', 'min_area': 3, 'max_area': 50, 'threshold': 100, 'denoise': 3, 'blocksize': 101, 'plot': False, 'plot_extracted': False, 'save_files': True}
Detected samples: 20
saving to files to /home/simone/development/python_università/fruit_detection/dataset_classification/Cetriolo
saved file: /home/simone/development/python_università/fruit_detection/dataset_classification/Cetriolo/cetrioli4_0_clean.tif
saved file: /home/simone/development/python_università/fruit_detection/dataset_classification/Cetriolo/cetrioli4_1_clean.tif
saved file: /home/simone/development/python_università/fruit_detection/dataset_classification/Cetriolo/cetrioli4_2_clean.tif
saved file: /home/simone/development/python_università/fruit_detection/dataset_classification/Cetriolo/cetrioli4_3_clean.tif
saved file: /home/simone/development/python_università/fruit_d

In [9]:
#generate csv for autokeras labels
# just an waful hack that works only 
import csv
csv_label = Path("/home/simone/development/python_università/fruit_detection/labels.csv")
list_label = []
for folder in [x for x in root_folder_save.iterdir() if x.is_dir()]:
    for file in folder.iterdir():
        list_label.append([str(file),folder.name]) # filename and label(folder name)
with open(csv_label, 'w') as f:
    writer = csv.writer(f)
    writer.writerows(list_label)